In [ ]:
!pip install gymnasium
!pip install pygame

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 8.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import gym
import math
from IPython import display

In [ ]:
# Setup the environment
env = gym.make("CartPole-v1")

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
alpha = 0.1
gamma = 0.9
num_episodes = 150000

epsilon = 0.9
epsilon_decay_value = 0.99995

total_reward = 0
prior_reward = 0

In [ ]:
Observation = [30, 30, 50, 50]
np_array_win_size = np.array([0.25, 0.25, 0.01, 0.1])

In [ ]:
q_matrix = np.random.uniform(low=0, high=1, size=(Observation + [env.action_space.n]))

In [ ]:
def get_discrete_state(state):
    discrete_state = state / np_array_win_size + np.array([15, 10, 1, 10])
    return tuple(discrete_state.astype(int))

In [ ]:
def epsilon_greedy_action_selection(epsilon, q_values):
    if np.random.random() < epsilon:
        return np.random.randint(0, len(q_values))
    else:
        return np.argmax(q_values)

In [ ]:
returns_sum = np.zeros(q_matrix.shape)
returns_count = np.zeros(q_matrix.shape)

In [ ]:
# Monte Carlo implementation

for episode in range(num_episodes +1):
    episode_transitions = []
    state = env.reset()
    done = False
    episode_reward = 0  # Store reward per episode

    while not done:
        state_index = get_discrete_state(state)
        action = epsilon_greedy_action_selection(0.1, q_matrix[state_index])
        next_state, reward, done, _ = env.step(action)
        episode_transitions.append((state_index, action, reward))
        episode_reward += reward
        state = next_state

    # Accumulate rewards and update average
    total_reward += episode_reward

    # Monte Carlo update at the end of the episode
    G = 0
    for transition in reversed(episode_transitions):
        state, action, reward = transition
        G = reward + gamma * G
        returns_sum[state + (action,)] += G
        returns_count[state + (action,)] += 1
        q_matrix[state + (action,)] = returns_sum[state + (action,)] / returns_count[state + (action,)]

    # Calculate and print average reward every 'print_every' episodes
    if episode % 100 == 0:
        mean_reward = total_reward / 100
        print(f"Episode: {episode}, Mean Reward: {mean_reward}, epsilon: {epsilon}")
        total_reward = 0
        if(mean_reward > 195.0):
          print("Problem Solved!")
          break
        total_reward = 0  # Reset total rewards after printing

Episode: 0, Mean Reward: 0.22, epsilon: 0.9
Episode: 100, Mean Reward: 22.67, epsilon: 0.9
Episode: 200, Mean Reward: 31.21, epsilon: 0.9
Episode: 300, Mean Reward: 31.56, epsilon: 0.9
Episode: 400, Mean Reward: 35.42, epsilon: 0.9
Episode: 500, Mean Reward: 37.19, epsilon: 0.9
Episode: 600, Mean Reward: 41.44, epsilon: 0.9
Episode: 700, Mean Reward: 51.26, epsilon: 0.9
Episode: 800, Mean Reward: 55.1, epsilon: 0.9
Episode: 900, Mean Reward: 72.72, epsilon: 0.9
Episode: 1000, Mean Reward: 71.82, epsilon: 0.9
Episode: 1100, Mean Reward: 76.94, epsilon: 0.9
Episode: 1200, Mean Reward: 91.77, epsilon: 0.9
Episode: 1300, Mean Reward: 86.43, epsilon: 0.9
Episode: 1400, Mean Reward: 99.54, epsilon: 0.9
Episode: 1500, Mean Reward: 95.92, epsilon: 0.9
Episode: 1600, Mean Reward: 99.69, epsilon: 0.9
Episode: 1700, Mean Reward: 99.25, epsilon: 0.9
Episode: 1800, Mean Reward: 107.36, epsilon: 0.9
Episode: 1900, Mean Reward: 105.81, epsilon: 0.9
Episode: 2000, Mean Reward: 110.74, epsilon: 0.9
Epi

In [ ]:
# Q_Learning implementation

for episode in range(num_episodes + 1):
    state_index = get_discrete_state(env.reset())
    done = False
    episode_reward = 0

    while not done:
        if np.random.random() > epsilon:
            action = np.argmax(q_matrix[state_index])
        else:
            action = np.random.randint(0, env.action_space.n)

        new_state, reward, done, _ = env.step(action)
        episode_reward += reward
        new_state_index = get_discrete_state(new_state)

        if not done:
            max_next_q = np.max(q_matrix[new_state_index])
            current_q = q_matrix[state_index + (action,)]
            error = reward + gamma * (max_next_q - current_q)
            new_q = q_matrix[state_index + (action,)] + alpha * error
            q_matrix[state_index + (action,)] = new_q

        state_index = new_state_index

    if epsilon > 0.05:
        if episode_reward > prior_reward and episode > 100:
            epsilon = math.pow(epsilon_decay_value, episode - 100)

    total_reward += episode_reward
    prior_reward = episode_reward

    if episode % 100 == 0:
        mean_reward = total_reward / 100
        print(f"Episode: {episode}, Mean Reward: {mean_reward}, epsilon: {epsilon}")
        total_reward = 0
        if(mean_reward > 195.0):
          print("Problem Solved!")
          break

Episode: 0, Mean Reward: 1.66, epsilon: 0.2850669038119228
Episode: 100, Mean Reward: 189.15, epsilon: 0.2850669038119228
Episode: 200, Mean Reward: 29.88, epsilon: 0.9950123548119847
Episode: 300, Mean Reward: 24.15, epsilon: 0.9900495862284909
Episode: 400, Mean Reward: 23.58, epsilon: 0.9851115701738419
Episode: 500, Mean Reward: 21.94, epsilon: 0.9802471955509836
Episode: 600, Mean Reward: 24.76, epsilon: 0.9753093024395111
Episode: 700, Mean Reward: 22.29, epsilon: 0.9705418574497624
Episode: 800, Mean Reward: 22.84, epsilon: 0.9656045713250361
Episode: 900, Mean Reward: 24.22, epsilon: 0.9607884783313412
Episode: 1000, Mean Reward: 24.91, epsilon: 0.9560920131117725
Episode: 1100, Mean Reward: 24.54, epsilon: 0.9512282354250458
Episode: 1200, Mean Reward: 26.78, epsilon: 0.9464838464939238
Episode: 1300, Mean Reward: 23.19, epsilon: 0.9418573042672076
Episode: 1400, Mean Reward: 24.18, epsilon: 0.9370659405932598
Episode: 1500, Mean Reward: 26.11, epsilon: 0.9323921881638068
Epis

In [ ]:
# SARSA Learning

for episode in range(num_episodes + 1):
    state_index = get_discrete_state(env.reset())
    done = False
    episode_reward = 0

    # Choose action using epsilon-greedy strategy
    if np.random.random() > epsilon:
        action = np.argmax(q_matrix[state_index])
    else:
        action = np.random.randint(0, env.action_space.n)

    while not done:
        new_state, reward, done, _ = env.step(action)
        episode_reward += reward

        new_state_index = get_discrete_state(new_state)

        # Choose the next action (again using epsilon-greedy)
        if np.random.random() > epsilon:
            new_action = np.argmax(q_matrix[new_state_index])
        else:
            new_action = np.random.randint(0, env.action_space.n)

        # SARSA update
        current_q = q_matrix[state_index + (action,)]
        next_q = q_matrix[new_state_index + (new_action,)]
        error = reward + gamma * (next_q - current_q)
        new_q = q_matrix[state_index + (action,)] + alpha * error
        q_matrix[state_index + (action,)] = new_q

        state_index = new_state_index
        action = new_action

    # Epsilon decay
    if epsilon > 0.05:
        if episode_reward > prior_reward and episode > 100:
            epsilon = math.pow(epsilon_decay_value, episode - 100)

    total_reward += episode_reward
    prior_reward = episode_reward


    if episode % 100 == 0:
        mean_reward = total_reward / 100
        print(f"Episode: {episode}, Mean Reward: {mean_reward}, epsilon: {epsilon}")
        total_reward = 0
        if(mean_reward > 195.0):
          print("Problem Solved!")
          break

Episode: 0, Mean Reward: 2.22, epsilon: 0.22425143588801186
Episode: 100, Mean Reward: 190.64, epsilon: 0.22425143588801186
Episode: 200, Mean Reward: 25.33, epsilon: 0.9950621079173806
Episode: 300, Mean Reward: 23.42, epsilon: 0.9901981085184066
Episode: 400, Mean Reward: 24.37, epsilon: 0.9851115701738419
Episode: 500, Mean Reward: 24.81, epsilon: 0.9801981831912061
Episode: 600, Mean Reward: 22.86, epsilon: 0.9753093024395111
Episode: 700, Mean Reward: 24.85, epsilon: 0.9704448056903722
Episode: 800, Mean Reward: 23.0, epsilon: 0.9656528539677345
Episode: 900, Mean Reward: 22.27, epsilon: 0.9608365201573491
Episode: 1000, Mean Reward: 24.82, epsilon: 0.9559964063006914
Episode: 1100, Mean Reward: 23.87, epsilon: 0.9512757992150065
Episode: 1200, Mean Reward: 25.22, epsilon: 0.9464838464939238
Episode: 1300, Mean Reward: 24.8, epsilon: 0.9417631208914241
Episode: 1400, Mean Reward: 23.96, epsilon: 0.9371127962330714
Episode: 1500, Mean Reward: 23.7, epsilon: 0.9323921881638068
Episo

In [ ]:
env.close()